<a href="https://colab.research.google.com/github/giisg/QQAQtest/blob/main/QQAQ_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
from time import strftime
from datetime import datetime, timedelta
import urllib.request
#from apscheduler.schedulers.blocking import BlockingScheduler
import time
import requests
import numpy as np
#寫入DB部分先以註解表示

#抓縣市溫度與濕度的資料 
def crawl_weather():
    weather = []
    #板橋代表新北、新屋代表桃園、苗栗國代表國三、田中代表彰化、日月潭代表南投、古坑代表雲林、永康代表臺南、恆春代表屏東、國一N250K代表嘉義縣、新竹也代表新竹市，因為沒有新竹市的測站
    county = ['基隆','臺北','板橋','新屋','新竹','新竹','國三S156K','臺中','田中','日月潭','古坑','嘉義','國一N250K','永康','高雄','恆春','宜蘭','花蓮','臺東','澎湖','金門','馬祖']
    r = requests.get("https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-5702404A-5F4B-4CFE-8162-C11C29E66532&format=JSON&elementName=")
    a=json.loads(r.text)
    for i in range(len(a['records']['location'])):
        city=a['records']['location'][i]['parameter'][0]['parameterValue']
        label=a['records']['location'][i]['locationName']
        temp=a['records']['location'][i]['weatherElement'][3]['elementValue']#溫度
        humid=a['records']['location'][i]['weatherElement'][4]['elementValue']#濕度
        if float(temp) == -99:
            temp = np.nan
        if float(humid) == -99:
            humid = np.nan
        for j in range(len(county)):
          if label==county[j]:
            if j == 5:
              city = '新竹市'
            elif j == 12:
              city = '嘉義縣'
            #print(j,city,temp,humid)
            weather.append({'county':city, 'humid':humid, 'temp':temp})
    return weather

#抓縣市是否下雨的資料    
def crawl_Rain():
    rainL = []
    #板橋代表新北、田中代表彰化、西螺代表雲林、太保代表嘉義縣、新竹市東區代表新竹市、瑞穗代表花蓮、竹東代表新竹縣
    county = ['基隆','臺北','板橋','桃園','新竹市東區','竹東','苗栗','臺中','田中','南投','西螺','嘉義','太保','臺南','高雄','屏東','宜蘭','瑞穗','臺東','澎湖','金門','馬祖']
    r = requests.get("https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0002-001?Authorization=CWB-5702404A-5F4B-4CFE-8162-C11C29E66532&format=JSON", verify=False)
    #api_key可能須定期更新
    a=json.loads(r.text)
    for i in range(len(a['records']['location'])):
        label=a['records']['location'][i]['locationName']
        rain=a['records']['location'][i]['weatherElement'][1]['elementValue']#雨量
        city=a['records']['location'][i]['parameter'][0]['parameterValue']#城市
        if float(rain)< 0:
            if float(rain) == -998.0:
                rain = 0
            else:
                rain = np.nan
        else:
            rain = 1
        for j in range(len(county)):
          if label==county[j]:
            #print(j+1,city,rain)
            rainL.append({'county':city, 'rain':rain})
    return rainL

#抓除了THC、NMHC的所有資料
def crawl_EPA():
    r = requests.get("https://data.epa.gov.tw/api/v1/aqx_p_432?limit=1000&api_key=9be7b239-557b-4c10-9775-78cadfc555e9&format=json", verify=False)
    #api_key可能須定期更新
    a=json.loads(r.text)
    #print(a)
    Y=datetime.now().year
    M=datetime.now().month
    D=datetime.now().day
    H=datetime.now().hour
    site_feature = ['PM2.5','PM10','O3','CO','SO2','NO2','WindSpeed','WindDirec']#api中無NMHC
    EPA_list = []
    #client = MongoClient('140.116.164.187',27017)
    #client.test.authenticate("ryuk","kidlab95400")
    #db = client.AirP
    #coll = db.record3
    for i in range(len(a['records'])):#搜尋每個測站
        label=a['records'][i]
        for j in range(len(site_feature)):#依照feature找value
            FN=site_feature[j]
            hr_data_dict = {'hr_v':label[FN], 'hr_t': int(H), 'hr_flag': 0}
            #query = {'feature': site_feature[j], "id": str(site_list[i]), "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #fp.write(site_feature[j]+'\n')
            #if coll.find_one(query) == None:
            mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': FN, 'id': label['SiteId'], 'hr_data': [hr_data_dict], 
                      'county': label['County'], 'rain': 0, 'humid':0 , 'temp':0}
            EPA_list.append(mydict)
    return EPA_list
            #x = coll.insert_one(mydict)  
            #fp.write('Station_ID: '+str(site_list[i])+' ')
            #fp.write(str(x)+'\n')
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
                #fp.write('Station_ID: '+str(site_list[i])+' ')
                #fp.write(str(x)+'\n')
            
            #coll = db.record5
            #query = {'feature': site_feature[j], "id": i+1, "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #if coll.find_one(query) == None:
                #mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': FN, 'id': i+1, 'hr_data': [hr_data_dict]}
                #x = coll.insert_one(mydict)  
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
                
#抓NMHC的資料                     
def crawl_EPA_NMHC():
    r = requests.get("https://data.epa.gov.tw/api/v1/aqx_p_313?offset=0&limit=1000&api_key=d695558f-7f7d-453e-97fb-fc392a55c0a6", verify=False)
    #api_key可能須定期更新
    a=json.loads(r.text)
    #print(a)
    Y=datetime.now().year
    M=datetime.now().month
    D=datetime.now().day
    H=datetime.now().hour
    site_feature = ['Concentration']#NMHC
    NMHC_list = []
    #client = MongoClient('140.116.164.187',27017)
    #client.test.authenticate("ryuk","kidlab95400")
    #db = client.AirP
    #coll = db.record3
    for i in range(len(a['records'])):#搜尋每個測站
        label=a['records'][i]
        for j in range(len(site_feature)):#依照feature找value
            FN=site_feature[j]
            hr_data_dict = {'hr_v':label[FN], 'hr_t': int(H), 'hr_flag': 0}
            #query = {'feature': site_feature[j], "id": str(site_list[i]), "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #fp.write(site_feature[j]+'\n')
            #if coll.find_one(query) == None:
            mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': 'NMHC', 'id': label['SiteId'], 'hr_data': [hr_data_dict],
                      'county': label['County'], 'rain': 0, 'humid':0 , 'temp':0}
            NMHC_list.append(mydict)
    return  NMHC_list
            #x = coll.insert_one(mydict)  
            #fp.write('Station_ID: '+str(site_list[i])+' ')
            #fp.write(str(x)+'\n')
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
                #fp.write('Station_ID: '+str(site_list[i])+' ')
                #fp.write(str(x)+'\n')
            
            #coll = db.record5
            #query = {'feature': site_feature[j], "id": i+1, "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #if coll.find_one(query) == None:
                #mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': FN, 'id': i+1, 'hr_data': [hr_data_dict]}
                #x = coll.insert_one(mydict)  
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
                
#抓THC的資料                
def crawl_EPA_THC():
    r = requests.get("https://data.epa.gov.tw/api/v1/aqx_p_312?offset=0&limit=1000&api_key=d695558f-7f7d-453e-97fb-fc392a55c0a6", verify=False)
    #api_key可能須定期更新
    a=json.loads(r.text)
    #print(a)
    Y=datetime.now().year
    M=datetime.now().month
    D=datetime.now().day
    H=datetime.now().hour
    site_feature = ['Concentration']#THC
    THC_list = []
    #client = MongoClient('140.116.164.187',27017)
    #client.test.authenticate("ryuk","kidlab95400")
    #db = client.AirP
    #coll = db.record3
    for i in range(len(a['records'])):#搜尋每個測站
        label=a['records'][i]
        for j in range(len(site_feature)):#依照feature找value
            FN=site_feature[j]
            hr_data_dict = {'hr_v':label[FN], 'hr_t': int(H), 'hr_flag': 0}
            #query = {'feature': site_feature[j], "id": str(site_list[i]), "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #fp.write(site_feature[j]+'\n')
            #if coll.find_one(query) == None:
            mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': 'THC', 'id': label['SiteId'], 'hr_data': [hr_data_dict], 
                      'county': label['County'],'rain': 0, 'humid':0 , 'temp':0}
            THC_list.append(mydict)
    return  THC_list[0:38]
            #x = coll.insert_one(mydict)  
            #fp.write('Station_ID: '+str(site_list[i])+' ')
            #fp.write(str(x)+'\n')
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
                #fp.write('Station_ID: '+str(site_list[i])+' ')
                #fp.write(str(x)+'\n')
            
            #coll = db.record5
            #query = {'feature': site_feature[j], "id": i+1, "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
            #if coll.find_one(query) == None:
                #mydict = {'date': {'year': int(Y), 'month': int(M), 'day': int(D)}, 'station': label['SiteName'], 'feature': FN, 'id': i+1, 'hr_data': [hr_data_dict]}
                #x = coll.insert_one(mydict)  
            #else:
                #hr_data_list = coll.find_one(query)['hr_data']
                #hr_data_list.append(hr_data_dict)
                #newvalue = {"$set": {"hr_data" : hr_data_list}}
                #x = coll.update_one(query,newvalue)
               
#寫入DB             
def intoDB(result):
  client = MongoClient('140.116.164.187',27017)
  client.test.authenticate("ryuk","kidlab95400")
  db = client.AirP
  coll = db.record3
  for i in range(len(result)):
    if coll.find_one(query) == None:
      query = {'feature': result[i]['feature'], "id": result[i]['id'], "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
      fp.write(result[i]['feature']+'\n')
      x = coll.insert_one(result[i])
      fp.write('Station_ID: '+str(result[i]['id'])+' ')
      fp.write(str(x)+'\n')
    else:
      hr_data_list = coll.find_one(query)['hr_data']
      hr_data_list.append(result[i]['hr_data'])
      newvalue = {"$set": {"hr_data" : hr_data_list}}
      x = coll.update_one(query,newvalue)
      fp.write('Station_ID: '+str(result[i]['id'])+' ')
      fp.write(str(x)+'\n')
            
      coll = db.record5
      query = {'feature': result[i]['feature'], "id": result[i]['id'], "date.year": int(year),"date.month": int(mon), "date.day": int(date)}
      if coll.find_one(query) == None:
        x = coll.insert_one(result[i])  
      else:
        hr_data_list = coll.find_one(query)['hr_data']
        hr_data_list.append(result[i]['hr_data'])
        newvalue = {"$set": {"hr_data" : hr_data_list}}
        x = coll.update_one(query,newvalue)
        
#合併環保局所有汙染源的資料        
def Mixall(EPA,THC,NMHC):
  THC_list = []
  NMHC_list = []
  all = EPA
  print(len(all))
  count_THC = 0
  count_NMHC = 0
  for i in range(len(THC)):
    for j in range(len(EPA)):
      count_THC += 1
      if THC[i]['id'] == EPA[j]['id'] and count_THC % 8 == 0:
        all.append(THC[i])
        THC_list.append(THC[i])
  for i in range(len(NMHC)):
    for j in range(len(EPA)):
      count_NMHC += 1
      if NMHC[i]['id'] == EPA[j]['id'] and count_NMHC % 8 == 0:
        all.append(NMHC[i])
        NMHC_list.append(NMHC[i])
      # count_NMHC += 1
  print(len(all))
  print(len(THC_list))
  print(len(NMHC_list))
  return all

#合併環保局跟氣象局的資料
def Mixall_weather(all,rr,ww):
  for i in range(len(rr)):
    for j in range(len(all)):
      if rr[i]['county'] == all[j]['county']:
        all[j]['rain'] = rr[i]['rain']
        all[j]['humid'] = ww[i]['humid']
        all[j]['temp'] = ww[i]['temp'] 
  return all                   
# EPA = crawl_EPA()
# THC = crawl_EPA_THC()
# NMHC = crawl_EPA_NMHC()
# rr = crawl_Rain()
# ww = crawl_weather()
# all = Mixall(crawl_EPA(),crawl_EPA_THC(),crawl_EPA_NMHC())
result = Mixall_weather(Mixall(crawl_EPA(),crawl_EPA_THC(),crawl_EPA_NMHC()),rr,ww)

#放入DB跟每隔一小時跑資料用的
#intoDB(Mixall_weather(Mixall(crawl_EPA(),crawl_EPA_THC(),crawl_EPA_NMHC()),rr,ww))
#sched = BlockingScheduler()
#sched.add_job(intoDB, 'interval', minutes = 60)
#sched.start()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


672
752
38
42


In [ ]:
#測試用的
for i in range(len(result)):
  print(result[i])

{'date': {'year': 2021, 'month': 1, 'day': 6}, 'station': '基隆', 'feature': 'PM2.5', 'id': '1', 'hr_data': [{'hr_v': '12', 'hr_t': 2, 'hr_flag': 0}], 'county': '基隆市', 'rain': 1, 'humid': '0.71', 'temp': '17.80'}
{'date': {'year': 2021, 'month': 1, 'day': 6}, 'station': '基隆', 'feature': 'PM10', 'id': '1', 'hr_data': [{'hr_v': '29', 'hr_t': 2, 'hr_flag': 0}], 'county': '基隆市', 'rain': 1, 'humid': '0.71', 'temp': '17.80'}
{'date': {'year': 2021, 'month': 1, 'day': 6}, 'station': '基隆', 'feature': 'O3', 'id': '1', 'hr_data': [{'hr_v': '41.2', 'hr_t': 2, 'hr_flag': 0}], 'county': '基隆市', 'rain': 1, 'humid': '0.71', 'temp': '17.80'}
{'date': {'year': 2021, 'month': 1, 'day': 6}, 'station': '基隆', 'feature': 'CO', 'id': '1', 'hr_data': [{'hr_v': '0.28', 'hr_t': 2, 'hr_flag': 0}], 'county': '基隆市', 'rain': 1, 'humid': '0.71', 'temp': '17.80'}
{'date': {'year': 2021, 'month': 1, 'day': 6}, 'station': '基隆', 'feature': 'SO2', 'id': '1', 'hr_data': [{'hr_v': '1', 'hr_t': 2, 'hr_flag': 0}], 'county': '基隆